In [1]:
import moviepy.editor as mp
clip = mp.VideoFileClip("F:/prism/New folder/production_id_4804794 (1440p).mp4")
clip_resized = clip.resize(height=720) # make the height 720px ( According to moviePy documenation The width is then computed so that the width/height ratio is conserved.)
clip_resized.write_videofile("F:/prism/New folder/try2/output_video.mp4")


Moviepy - Building video F:/prism/New folder/try2/output_video.mp4.
Moviepy - Writing video F:/prism/New folder/try2/output_video.mp4



Moviepy - Done !
Moviepy - video ready F:/prism/New folder/try2/output_video.mp4


In [2]:
from IPython.display import display, Javascript

# Use the display and Javascript function to run JavaScript code to restart the kernel
display(Javascript('Jupyter.notebook.kernel.restart()'))


<IPython.core.display.Javascript object>

In [3]:
import cv2
import os
import shutil
from moviepy.editor import VideoFileClip

# Function to calculate the frame rate of a video
def get_frame_rate(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise Exception("Error: Could not open video file.")
    
    frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
    cap.release()
    return frame_rate


part = 120


# Function to split a video into 60-frame parts and save them
def split_and_save_video(video_path, output_dir):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise Exception("Error: Could not open video file.")
    
    frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    part_length = part # Number of frames in each part
    current_part = 1

    while True:
        success, frame = cap.read()
        if not success:
            break

        output_part_path = os.path.join(output_dir, f"part_{current_part}.mp4")
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_part_path, fourcc, frame_rate, (frame.shape[1], frame.shape[0]))

        for _ in range(part_length):
            if not success:
                break
            out.write(frame)
            success, frame = cap.read()
        
        out.release()
        current_part += 1

    cap.release()
    return current_part

# Input video file path
video_path = 'F:/prism/New folder/try2/output_video.mp4'

# Output directory for saving parts
output_directory = 'F:/prism/New folder/try1'
if os.path.exists(output_directory):
    shutil.rmtree(output_directory)
os.makedirs(output_directory, exist_ok=True)


frame_rate = get_frame_rate(video_path)
num_parts = split_and_save_video(video_path, output_directory)

print(f"Video frame rate: {frame_rate}")
print(f"Video split into {num_parts - 1} parts and saved in '{output_directory}'.")

from towhee import ops, pipe, DataCollection
from glob import glob

# Define the Towhee pipeline
p = (
    pipe.input('video_path_root')
    .flat_map('video_path_root', 'video_path', glob)
    .map('video_path', 'frames', ops.video_decode.ffmpeg(sample_type='uniform_temporal_subsample', args={'num_samples': 32}))
    .map('frames', ('predicts', 'scores', 'features'), ops.action_classification.pytorchvideo(model_name='mvit_base_32x3', skip_preprocess=True, topk=5))
    .output('video_path', 'predicts', 'scores')
)

#DataCollection(p('./try1/*.mp4')).show()
# Create DataCollection and process video parts
data_collection = DataCollection(p('./try1/*.mp4'))

# Extract the top predicted actions for each video part
top_actions = []

for result in data_collection:
    predicts = result['predicts'][0] if result['predicts'] else []
    scores = result['scores'][0] if result['scores'] else []
    
    # # Filter actions with scores greater than 0.5
    # filtered_actions = [predicts[i] if (scores and scores[i] > 0.5) else None for i in range(len(predicts))]
    
    # top_actions.append(filtered_actions)
    top_actions = [result['predicts'][0] if result['scores'][0] > 0.5 else [] for result in data_collection]


# Print the top predicted actions for each video part
# for i, actions in enumerate(top_actions, start=1):
#     print(f"Top predicted actions for Part {i}: {actions}")


# Calculate start and end times for actions and merge consecutive actions with the same label
def calculate_timestamps(top_actions, frame_rate):
    timestamps = []
    current_action = None
    start_time = 0

    for i, actions in enumerate(top_actions):
        # if not actions:
        #     current_action = None
        #     continue

        if not current_action:
            current_action = actions
            # print(i)
            start_time = (i)*part / frame_rate
            
        elif current_action == actions:
            continue
        else:
            end_time = (i)*part / frame_rate
            timestamps.append((current_action, start_time, end_time))
            current_action = actions
            start_time = (i)*part / frame_rate

    # Add the last action if it exists
    video_clip = VideoFileClip(video_path)
    video_duration = video_clip.duration
    video_clip.close()
    if current_action:
        end_time = len(top_actions) *part / frame_rate
        timestamps.append((current_action, start_time, video_duration))

    return timestamps

# Calculate timestamps based on the top predicted actions
frame_rate = get_frame_rate(video_path)
timestamps = calculate_timestamps(top_actions, frame_rate)

for action,start,end in timestamps:
    print(f"Action: {action}, Start Time: {start:.2f}, End time: {end:.2f}")




Video frame rate: 25
Video split into 3 parts and saved in 'F:/prism/New folder/try1'.


2023-11-06 17:11:11,270 - 4380 - warnings.py-warnings:109 - WARNING: c:\Users\Vedant\miniconda3\Lib\site-packages\torchvision\transforms\functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(

Using cache found in C:\Users\Vedant/.cache\torch\hub\facebookresearch_pytorchvideo_main


Action: push up, Start Time: 0.00, End time: 11.64
